<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_wind_FTP_data_access.ipynb"><< Accessing OSI SAF wind products from KNMI FTP server</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2a_OSI_SAF_wind_ASCAT_file_structure.ipynb">OSI SAF wind ASCAT file structure >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT <br>

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-wind/HEAD?labpath=1_OSI_SAF_wind_introductory%2F1_1_OSI_SAF_wind_FTP_data_access.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF wind: Introductory - Metop ASCAT wind products</h3></div>

# 1.1 OSI SAF advanced data access via the Data Store using eumdac

### Data used

| Dataset | EUMETSAT collection ID | OSI SAF website description | OSI SAF identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Metop-A ASCAT L2 25 km Winds Data Record | EO:EUM:DAT:METOP:OSI-150-A | <a href="https://osi-saf.eumetsat.int/products/osi-150-a" target="_blank">Description</a> | OSI-150-a | 
| Metop-A ASCAT L2 12.5 km Winds Data Record | EO:EUM:DAT:METOP:OSI-150-b | <a href="https://osi-saf.eumetsat.int/products/osi-150-b" target="_blank">Description</a> | OSI-150-b | 
| Metop-B ASCAT coastal Winds | EO:EUM:DAT:METOP:OSI-104-b | <a href="https://osi-saf.eumetsat.int/products/osi-104-b" target="_blank">Description</a> | OSI-104-b |
| Metop-C ASCAT coastal Winds | EO:EUM:DAT:METOP:OSI-104-c | <a href="https://osi-saf.eumetsat.int/products/osi-104-c" target="_blank">Description</a> | OSI-104-c ||

### Learning outcomes

At the end of this notebook you will know;
* How to download Metop ASCAT winds from the Eumetsat Data Store

### Outline


Each product package includes:

* measurement data files (NetCDF-4 format)

<div class="alert alert-info" role="alert">
    
## <a id='TOC-TOP'></a>Contents

</div>

1. [Creating the workspace](#section1)
1. [Setting login information](#section2)
1. [Example 3: Filter by space and time](#section3)
1. [Example 4: Download by component](#section4)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [8]:
import os                       # a library that allows us access to basic operating system commands
import json                     # a library that helps us make JSON format files
import datetime                 # a libary that allows us to work with dates and times
import shutil                   # a library that allows us access to basic operating system commands like copy
import xarray as xr             # a library that supports the use of multi-dimensional arrays in Python
import matplotlib.pyplot as plt # a library that support plotting
import numpy as np              # a library that lets us work with arrays; we import this with a new name "np"
import eumdac                   # a tool that helps us download via the eumetsat/data-store
import zipfile                  # a library that support zip file
from pathlib import Path        # a library that helps construct system path objects

Next we will create directory to store the products we will download in this notebook.

In [10]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

To access OSI SAF from the <a href="https://data.eumetsat.int">EUMETSAT Data Store</a>. we will use the EUMETSAT Data Access Client (`eumdac`). If you are working with the recommended Anaconda Python distribution and used the supplied environment file (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can install eumdac using;

`conda install -c eumetsat eumdac`

However, you can also find the source code on the <a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac">EUMETSAT gitlab</a>. Please visit the EUMETSAT user support confluence spaces for the the <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DSDS/overview">Data Store</a> and <a href="https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/overview">eumdac</a> for more information.

In order to allow us to download data from the Data Store via API, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumdac/credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumdac/credentials` file in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system. In side this folder we need to create a folder called `.eumdac` (the dot is important) and within that a file called `credentials` (with no extension).

In this file we need to add the following information exactly as follows;

```
<your_consumer_key>,<your_consumer_secret>
```

You must replace `<your_consumer_key>` and `<your_consumer_secret>` with the information you extract from https://api.eumetsat.int/api-key/. You will need a <a href="https://eoportal.eumetsat.int/">EUMETSAT Earth Observation Portal account</a> to access this link, and in order to see the information you must click the "Show hidden fields" button at the bottom of the page. <b>To help with this step, we have made a supporting notebook that will create this file for you. You can find it <a href="../working-with-python/API_authentication.ipynb">here</a>, provided you cloned this repository with it's submodules, as detailed in the README.</b>

*Note: your key and secret are permanent, so you only need to do this once, but you should take care to never share them*

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

In [12]:
# load credentials
credentials_file = os.path.join(os.path.expanduser("~"),'.eumdac','credentials')
credentials = Path(credentials_file).read_text().split(',')
token = eumdac.AccessToken((credentials[0], credentials[1]))
print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '5c5eee11-8920-3c33-af1f-7a40a11874b1' expires 2025-06-12 21:33:37.130597


#### Option 2: provide credentials directly

You can provide your credentials directly as follows;

`token = eumdac.AccessToken((consumer_key, consumer_secret))`

*Note: this method is convenient in the short term, but is not really recommended as you have to put your key and secret in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

Once we have a token (by either method described above) we can create a **datastore** 'object' that contains all the collections, and filter this list for those that are relevant to us. The code to the right of the colon is what will tell use the ID for the collection we are interested in.

We will work with the Metop-A ASCAT L2 25 km wind collection and Metop-A ASCAT L2 12.5 km wind collection throughout this notebook. The code for these are `EO:EUM:DAT:METOP:OSI-150-A` and `EO:EUM:DAT:METOP:OSI-150-B`

In [15]:
# set collection ID for ASCAT L2 25km winds data record
collectionID_1 = 'EO:EUM:DAT:METOP:OSI-150-A'
# set collection ID for ASCAT L2 12.5km winds data record
collectionID_2 = 'EO:EUM:DAT:METOP:OSI-150-B'
# set collection ID for Metop ASCAT costal wind
collectionID_3 = 'EO:EUM:DAT:METOP:OSI-104'

<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

#### Metop A ASCAT 25 km wind record
Data available until 2014-04-01

Let's start by creating a specific folder to hold the Metop-A/Ascat 25 km Wind products

In [19]:
download_dir = os.path.join(os.getcwd(), "products", "ascat_a", "25")
os.makedirs(download_dir, exist_ok=True)

We can filter by time using start date and end date 

In [21]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID_1)
# Select start and end date for search
start = datetime.datetime(2014, 3, 31,0,0) # date format YYYY, MM, DD, HH, mm
end = datetime.datetime(2014, 3, 31, 6, 0)  # date format YYYY, MM, DD, HH, mm

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method.

In [23]:
# For ASCAT 25km wind collection
roi = [[101.08,36.04], [166.61,36.04], [167.21,-7.19] ,[102.29,-7.95] ,[101.08,36.04]]

products = selected_collection.search(
        geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
        dtstart=start,
        dtend=end)

for product in products:
    print(product)

OR1ASW025_20140331_024611_38634_M02
OR1ASW025_20140331_010448_38633_M02
OR1ASW025_20140330_232326_38632_M02


Or filtering a square region into the **bbox** argument (usefull for square region)

In [25]:
# For ASCAT 25km wind collection
area = '97.94, -4.91, 171.03, 36.86' # longitude min, latitude min, longitude max, latitude max
products = selected_collection.search(
        bbox=area, 
        dtstart=start, 
        dtend=end)

for product in products:
        print(product)

OR1ASW025_20140331_024611_38634_M02
OR1ASW025_20140331_010448_38633_M02
OR1ASW025_20140330_232326_38632_M02


Downloading by component.\
It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. This method can be used to prune downloads for the enhanced, reduced or standard measurements only.

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

In [27]:
for product_id in products:
    print(f"Retrieving: {product_id}")    

    # selected_collection = datastore.get_collection(collection_id)
    selected_product = datastore.get_product(product_id=product_id, collection_id=collectionID_1)

    with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(selected_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

Retrieving: OR1ASW025_20140331_024611_38634_M02
Download of product OR1ASW025_20140331_024611_38634_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/25/OR1ASW025_20140331_024611_38634_M02.zip finished.
Retrieving: OR1ASW025_20140331_010448_38633_M02
Download of product OR1ASW025_20140331_010448_38633_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/25/OR1ASW025_20140331_010448_38633_M02.zip finished.
Retrieving: OR1ASW025_20140330_232326_38632_M02
Download of product OR1ASW025_20140330_232326_38632_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/25/OR1ASW025_20140330_232326_38632_M02.zip finished.


#### Metop A ASCAT 12.5 km wind data record
Data available until 2014-04-01

Let's start by creating a specific folder to hold the Metop-A/Ascat 12.5 km Wind products

In [30]:
download_dir = os.path.join(os.getcwd(), "products", "ascat_a", "12")
os.makedirs(download_dir, exist_ok=True)

We can filter by time using start date and end date 

In [32]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID_2)
# Select start and end date for search
start = datetime.datetime(2014, 3, 31,0,0) # date format YYYY, MM, DD, HH, mm
end = datetime.datetime(2014, 3, 31, 6, 0)  # date format YYYY, MM, DD, HH, mm

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method.

In [34]:
# For ASCAT 12.5km wind collection
roi = [[101.08,36.04], [166.61,36.04], [167.21,-7.19] ,[102.29,-7.95] ,[101.08,36.04]]
products = selected_collection.search(
        geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
        dtstart=start,
        dtend=end)

for product in products:
    print(product)

OR1ASWC12_20140331_024607_38634_M02
OR1ASWC12_20140331_010446_38633_M02
OR1ASWC12_20140330_232324_38632_M02


Or filtering a square region into the **bbox** argument (usefull for square region)

In [36]:
# For ASCAT 12.5km wind collection
area = '97.94, -4.91, 171.03, 36.86' # longitude min, latitude min, longitude max, latitude max
products = selected_collection.search(
        bbox=area, 
        dtstart=start, 
        dtend=end)

for product in products:
        print(product)

OR1ASWC12_20140331_024607_38634_M02
OR1ASWC12_20140331_010446_38633_M02
OR1ASWC12_20140330_232324_38632_M02


Downloading by component.\
It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. This method can be used to prune downloads for the enhanced, reduced or standard measurements only.

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

In [38]:
for product_id in products:
    print(f"Retrieving: {product_id}")    

    # selected_collection = datastore.get_collection(collection_id)
    selected_product = datastore.get_product(product_id=product_id, collection_id=collectionID_2)

    with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
        print(f'Downloading {fsrc.name}.')
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {fsrc.name} finished.')

    with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.startswith(str(selected_product)):
                zip_ref.extract(file, download_dir)
        print(f'Unzipping of product {fdst.name} finished.')

    os.remove(fdst.name)

Retrieving: OR1ASWC12_20140331_024607_38634_M02
Download of product OR1ASWC12_20140331_024607_38634_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/12/OR1ASWC12_20140331_024607_38634_M02.zip finished.
Retrieving: OR1ASWC12_20140331_010446_38633_M02
Download of product OR1ASWC12_20140331_010446_38633_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/12/OR1ASWC12_20140331_010446_38633_M02.zip finished.
Retrieving: OR1ASWC12_20140330_232324_38632_M02
Download of product OR1ASWC12_20140330_232324_38632_M02.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_a/12/OR1ASWC12_20140330_232324_38632_M02.zip finished.


#### Metop B/C coastal wind 

Let's start by creating a specific folder to hold the Metop-B and Metop-C/Ascat coastal Wind products

In [41]:
# Create a new directory for Metop B costal wind
download_dir1 = os.path.join(os.getcwd(), "products", "ascat_b")
os.makedirs(download_dir1, exist_ok=True)

# Create a new directory for Metop C costal wind
download_dir2 = os.path.join(os.getcwd(), "products", "ascat_c", "coa")
os.makedirs(download_dir2, exist_ok=True)

We can filter by time using start date and end date 

In [43]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID_3)
# Select start and end date for search
start = datetime.datetime(2023, 6, 13,0,0) # date format YYYY, MM, DD, HH, mm
end = datetime.datetime(2023, 6, 13, 6, 0)  # date format YYYY, MM, DD, HH, mm

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method.

In [45]:
roi = [[101.08,36.04], [166.61,36.04], [167.21,-7.19] ,[102.29,-7.95] ,[101.08,36.04]]
products = selected_collection.search(
        geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
        dtstart=start,
        dtend=end)

for product in products:
    print(product)

ascat_20230613_023900_metopc_23855_eps_o_coa_3301_ovw.l2
ascat_20230613_014800_metopb_55697_eps_o_coa_3301_ovw.l2
ascat_20230613_005700_metopc_23854_eps_o_coa_3301_ovw.l2
ascat_20230613_000600_metopb_55696_eps_o_coa_3301_ovw.l2
ascat_20230612_231800_metopc_23853_eps_o_coa_3301_ovw.l2
ascat_20230612_222400_metopb_55695_eps_o_coa_3301_ovw.l2


Or filtering a square region into the **bbox** argument (usefull for square region)

In [47]:
area = '97.94, -4.91, 171.03, 36.86' # longitude min, latitude min, longitude max, latitude max
products = selected_collection.search(
        bbox=area, 
        dtstart=start, 
        dtend=end)

for product in products:
        print(product)

ascat_20230613_032700_metopb_55698_eps_o_coa_3301_ovw.l2
ascat_20230613_023900_metopc_23855_eps_o_coa_3301_ovw.l2
ascat_20230613_014800_metopb_55697_eps_o_coa_3301_ovw.l2
ascat_20230613_005700_metopc_23854_eps_o_coa_3301_ovw.l2
ascat_20230613_000600_metopb_55696_eps_o_coa_3301_ovw.l2
ascat_20230612_231800_metopc_23853_eps_o_coa_3301_ovw.l2
ascat_20230612_222400_metopb_55695_eps_o_coa_3301_ovw.l2


Downloading by component.\
It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. This method can be used to prune downloads for the enhanced, reduced or standard measurements only.

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

Downloading Metop B data

In [50]:
for product_id in products:
    # Downloading data from Metop B
    if "metopb" in str(product_id):
        print(f"Retrieving: {product_id}")    
    
        # selected_collection = datastore.get_collection(collection_id)
        selected_product = datastore.get_product(product_id=product_id, collection_id=collectionID_3)

        with selected_product.open() as fsrc, open(os.path.join(download_dir1, fsrc.name), mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')

        with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith(str(selected_product)):
                    zip_ref.extract(file, download_dir1)
            print(f'Unzipping of product {fdst.name} finished.')
    
        os.remove(fdst.name)

Retrieving: ascat_20230613_032700_metopb_55698_eps_o_coa_3301_ovw.l2
Download of product ascat_20230613_032700_metopb_55698_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_b/ascat_20230613_032700_metopb_55698_eps_o_coa_3301_ovw.l2.zip finished.
Retrieving: ascat_20230613_014800_metopb_55697_eps_o_coa_3301_ovw.l2
Download of product ascat_20230613_014800_metopb_55697_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_b/ascat_20230613_014800_metopb_55697_eps_o_coa_3301_ovw.l2.zip finished.
Retrieving: ascat_20230613_000600_metopb_55696_eps_o_coa_3301_ovw.l2
Download of product ascat_20230613_000600_metopb_55696_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/intern

Downloading Metop C data

In [52]:
for product_id in products:
    # Downloading data from Metop C
    if "metopc" in str(product_id):
        print(f"Retrieving: {product_id}")    
    
        # selected_collection = datastore.get_collection(collection_id)
        selected_product = datastore.get_product(product_id=product_id, collection_id=collectionID_3)

        with selected_product.open() as fsrc, open(os.path.join(download_dir2, fsrc.name), mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')

        with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith(str(selected_product)):
                    zip_ref.extract(file, download_dir2)
            print(f'Unzipping of product {fdst.name} finished.')
    
        os.remove(fdst.name)

Retrieving: ascat_20230613_023900_metopc_23855_eps_o_coa_3301_ovw.l2
Download of product ascat_20230613_023900_metopc_23855_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_c/coa/ascat_20230613_023900_metopc_23855_eps_o_coa_3301_ovw.l2.zip finished.
Retrieving: ascat_20230613_005700_metopc_23854_eps_o_coa_3301_ovw.l2
Download of product ascat_20230613_005700_metopc_23854_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMTS/internal/ocean/sensors/learn-osi-saf-wind/1_OSI_SAF_wind_introductory/products/ascat_c/coa/ascat_20230613_005700_metopc_23854_eps_o_coa_3301_ovw.l2.zip finished.
Retrieving: ascat_20230612_231800_metopc_23853_eps_o_coa_3301_ovw.l2
Download of product ascat_20230612_231800_metopc_23853_eps_o_coa_3301_ovw.l2.zip finished.
Unzipping of product /Users/benloveday/Code/git_repositories/CMT

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_wind_FTP_data_access.ipynb"><< Accessing OSI SAF wind products from KNMI FTP server</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2a_OSI_SAF_wind_ASCAT_file_structure.ipynb">OSI SAF wind ASCAT file structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>